# Prova di esame (laboratorio)

La prova avrà una durata massima di due ore e porterà a una valutazione massima di 21 punti. Il voto finale si otterrà per somma del punteggio del test di teoria e della prova di laboratorio. 

La prova consiste in un insieme di quiz proposti attraverso un Jupyter Notebook (non sono ammessi altri strumenti software) da svolgere sui computer del laboratorio (non sono ammessi altri strumenti hardware). Durante la prova si può consultare il libro di testo, le slide usate a lezione, un manuale python (approvato dal docente) oppure le guide in linea suggerite da Jupyter Notebook. Non sono ammessi altri strumenti di consultazione. 

**Prima della prova**

Specificare la informazioni richieste nelle celle sottostanti e salvare il notebook con il proprio nome.

**Durante la prova**

Nelle celle markdown, cancellare il testo "YOUR ANSWER HERE" con la propria risposta. Nelle celle codice, rimuovere la linea di codice
```python
raise NotImplementedError()
```
e sostituirla con la propria implementazione.
Dopo ogni cella codice che deve contenere la risposta al quesito, esiste una cella codice con delle *asserzioni*, ossia delle istruzioni che verificano la correttezza della propria soluzione. (Queste asserzioni saranno usate per valutare la correttezza della risposta data e assegnare un punteggio.) E' possibile eseguire queste celle per verificare che la soluzione proposta sia corretta.


**Dopo la prova**

Salvare il file dopo aver terminato. Consegnare l'elaborato utilizzando la piattaforma ADA. 

## Dataset

### Dati Benderly e Zwick: Inflazione, crescita e rendimenti azionari
#### Descrizione

Dati delle serie temporali, 1952-1982. 

#### Formato

File CSV con 31 osservazioni su 5 variabili (più l'indice di riga):

- returns: rendimenti annuali reali delle azioni, misurati utilizzando la base di dati Ibbotson-Sinquefeld.
- growth: tasso di crescita annuale della produzione, misurato dal PIL reale (da un dato anno all'anno successivo).
- inflation: tasso d'inflazione, misurato come crescita del tasso dei prezzi (da dicembre dell'anno precedente a dicembre dell'anno in corso).
- growth2: tasso di crescita annuale del PNL reale dato da Baltagi.
- inflation2: tasso di inflazione come dato da Baltagi

    (Baltagi è il nome del ricercatore che ha studiato la serie temporale)


## Domande

### Caricamento dati

Definire un dataframe di nome `dataset` e popolarla con le righe del file `BenderlyZwick.csv`. Utilizzate la prima colonna come indice di riga (utilizzare il parametro `index_col`)

In [1]:
import pandas as pd

dataset = pd.read_excel('BenderlyZwick.xlsx' , index_col = 0)

dataset.head()

,returns,growth,inflation,growth2,inflation2
1,NaN,NaN,NaN,3.9,2.2
2,NaN,NaN,NaN,4.0,2.1
3,53.0,6.7,-0.4,-1.3,0.6
4,31.2,2.1,0.4,5.6,1.3
5,3.7,1.8,2.9,2.1,1.9


In [2]:
assert dataset.shape[0] == 31   # 31 osservazioni

In [3]:
assert dataset.shape[1] >= 5   # almeno 5 colonne

In [4]:
assert dataset.shape[1] == 5   # esattamente 5 colonne se la prima è usata come indice di riga

### Funzioni

Definire una funzione `rel_inc` con un solo argomento iterabile `it`. Se l'argomento rappresenta una sequenza di valori $a_0,a_1,\ldots, a_n$, allora la funzione restituisce una lista  costituita dalla sequenza $b_0, b_1, \ldots, b_n$ dove:
- $b_0$ è un valore `nan` (si ottiene con l'espressione `float('nan')`
- $b_i = \frac{b_i-b_{i-1}}{b_{i-1}}$ se $b_{i-1} \neq 0$, altrimenti $b_i=+\infty$, per $i=1,2,\ldots,n$ (il valore $+\infty$ si rappresenta con `float('+inf')`



In [5]:
import math

def rel_inc(it):
    
    return [
        float('nan') if i == 0 else (it[i] - it[i - 1]) / it[i - 1] if it[i - 1] != 0 else float('+inf') 
        for i in range(len(it))
    ]


rel_inc([1,2,0,3,4])

[nan, 1.0, -1.0, inf, 0.3333333333333333]

In [6]:
assert math.isnan(rel_inc([0])[0])   # il primo elemento deve essere NaN (si può testare solo con math.isnan)

In [7]:
assert rel_inc([1,2])[1] == 1.   # ultimo incremento del 100%

In [8]:
assert  rel_inc([1,2,0])[2] == -1.   # ultimo decremento del 100%

In [9]:
assert  rel_inc([1,2,0, 1])[3] == float('+inf')  # ultimo incremento infinito

In [10]:
assert len(rel_inc([1,2,0,3,4])) == len([1,2,0,3,4]) # la funzione preserva la lunghezza della lista

### Manipolazione di DataFrame

Aggiungere una colonna in `dataset` che contiene la differenza dei valori di `growth` meno `growth2`. La nuova colonna si chiamerà `delta_g`

In [11]:
dataset['delta_g'] = dataset.growth - dataset.growth2

dataset.head()

,returns,growth,inflation,growth2,inflation2,delta_g
1,NaN,NaN,NaN,3.9,2.2,NaN
2,NaN,NaN,NaN,4.0,2.1,NaN
3,53.0,6.7,-0.4,-1.3,0.6,8.0
4,31.2,2.1,0.4,5.6,1.3,-3.5
5,3.7,1.8,2.9,2.1,1.9,-0.3


In [12]:
assert 'delta_g' in dataset.columns

### Selezioni

Definire una variabile di nome `corr` valorizzata come segue. Se la media dei valori in `delta_g`, in valore assoluto, è minore di 0.1, allora `corr` sarà valorizzata alla *stringa* `'0'`, altrimenti se la media è positiva, allora `corr` sarà valorizzata a `'+'`, altrimenti a `'-'`. (Attenione: `delta_g` contiene valori mancanti)

In [13]:
corr = '0' if math.fabs(dataset.delta_g.mean()) <= 0 else '+' if dataset.delta_g.mean() > 0 else '-'

print(corr)

+


In [14]:
assert corr in '0+-'  # corr è un carattere valido

In [15]:
assert ord(corr) == 43 # corr è il carattere giusto

### Comprensioni di lista

Attraverso una comprensione di lista, definire una lista di nome `pn` con tanti elementi quante le righe del dataset. L'i-esimo elemento di `pn` sarà `+` se l'i-esimo valore della colonna `growth2` del dataset è maggiore o uguale a 0, altrimenti sarà `-`.

In [16]:
pn = ['+' if elem >= 0 else '-' for elem in dataset.growth2]

pn

['+',
 '+',
 '-',
 '+',
 '+',
 '+',
 '-',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '+',
 '-',
 '+',
 '+',
 '+',
 '-',
 '-',
 '+',
 '+',
 '+',
 '+',
 '-',
 '+',
 '-']

In [17]:
assert len(pn) == len(dataset)   # Stesso numero di righe

In [18]:
assert all(v in '+-' for v in pn)    # tutti gli elementi sono validi

### Concatenamento di stringhe

Attraverso i metodi di stringa, definire una stringa di none `spn` che consiste nel concatenamento di tutti i caratteri della lista `pn`.

In [19]:
spn = ''.join(elem for elem in pn)

print(spn)

++-+++-+++++++++++-+++--++++-+-


In [20]:
assert len(spn) == len(pn)   # Stessa lunghezza di pn

In [21]:
assert all(c in '+-' for c in spn)  # Caratteri validi 

In [22]:
from random import randrange

rand_i = randrange(len(spn))
assert spn[rand_i] == pn[rand_i]   # Valori corrispondenti tra spn e pn

### Espressioni regolari

Attraverso le espressioni regolari, sostituire le sequenze in `spn` costituite da un trattino, seguito da due o più segni '+' e seguiti da un altro trattino, con il carattere 'P'. Nel risultato, se tra una P e l'altra occorrono due o più simboli '+', sostituire tutta la sequenza dei '+' con una 'M'. Memorizzare il risultato nella variabile `red_spn`

In [23]:
import re

red_spn = re.sub(r'[P][+]{2,}[P]' , 'PMP' , re.sub(r'[-][+]{2,}[-]' , 'P' , spn))

In [24]:
assert all(c in '+-PM' for c in red_spn)   # caratteri validi

In [25]:
assert 'PMP' in red_spn    # seconda sostituzione corretta

In [26]:
assert '++P' in red_spn   # prima sostituzione corretta

### Dizionari

Definire un dizionario di nome `count` che associ, a ciascun carattere presente in `red_spn`, la relativa frequenza all'interno della stringa.

(Nota: se non si è risolto l'esercizio precedente, valorizzare `red_spn` a '-+PPMP++')

In [27]:
count = dict((elem , red_spn.count(elem)) for elem in red_spn)

print(count)

{'+': 3, 'P': 3, 'M': 1, '-': 1}


In [28]:
assert (count['+'] == count['P']) and (count['M'] == count['-']) # valori corretti

In [29]:
assert all(type(v)==int for v in count.values())   # tutti i valori sono interi